# Create meeting minutes from an audio or video file (OpenAI-only)

This notebook:
1. (Optional) extracts and normalizes audio with `ffmpeg`
2. splits audio into chunks for stable transcription
3. transcribes chunks using OpenAI Speech-to-Text
4. generates meeting minutes in Markdown using an OpenAI text model

> Prereq: set `OPENAI_API_KEY` in your environment.


In [1]:
import os
from getpass import getpass

if not os.getenv("OPENROUTER_API_KEY"):
    os.environ["OPENROUTER_API_KEY"] = getpass("Enter OPENROUTER_API_KEY: ")

print("OPENROUTER_API_KEY loaded:", bool(os.getenv("OPENROUTER_API_KEY")))


OPENROUTER_API_KEY loaded: True


In [2]:
%pip install -q --upgrade openai


Note: you may need to restart the kernel to use updated packages.


In [3]:
!winget install -e --id Gyan.FFmpeg
!ffmpeg -version



   - 
                                                                                                                        

  ███████████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  1024 KB / 2.65 MB
  ██████████████████████▒▒▒▒▒▒▒▒  2.00 MB / 2.65 MB
  ██████████████████████████████  2.65 MB / 2.65 MB
                                                                                                                        

  ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  0%
  ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  0%
  ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  1%
  ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  2%
  ▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  3%
  █▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  4%
  █▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  5%
  █▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  6%
  ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  7%
  ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  8%
  ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  9%
  ███▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  10%
  ███▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  11%
  ███▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  12%
  ███▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  13%
  ████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒  

In [4]:
import os, glob

matches = glob.glob(r"C:\Users\*\AppData\Local\Microsoft\WinGet\Packages\*\ffmpeg*\**\bin\ffmpeg.exe", recursive=True)
if matches:
    bin_dir = os.path.dirname(matches[0])
    if bin_dir not in os.environ.get("PATH", ""):
        os.environ["PATH"] = os.environ.get("PATH", "") + ";" + bin_dir

!ffmpeg -version


ffmpeg version 8.0.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --enab

In [ ]:
# Optional (terminal, one-time): uv add openai
# Keep this notebook cell as a no-op so it does not error in Python mode.


In [11]:
import os
from pathlib import Path
from openai import OpenAI
from IPython.display import Markdown, display

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

assert os.environ.get("OPENAI_API_KEY"), "Set OPENAI_API_KEY in your environment before running this notebook."


In [5]:
import os
from pathlib import Path
from openai import OpenAI
from IPython.display import Markdown, display

client = OpenAI(
    api_key=os.environ.get("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

assert os.environ.get("OPENROUTER_API_KEY"), "Set OPENROUTER_API_KEY in your environment before running this notebook."


## 1) Provide your input file

Set either `VIDEO_PATH` (to extract audio) **or** `AUDIO_PATH` (to transcribe directly).


In [12]:
# Option A: start from video
VIDEO_PATH = "Zeno Minutes.mp4"

# Option B: start from audio
AUDIO_PATH = None  # e.g. "denver_extract.mp3"

# Normalized audio for OpenAI transcription
SAFE_AUDIO = "audio_openai.mp3"
CHUNKS_DIR = Path("chunks")
CHUNK_SECONDS = 600  # 10 minutes

assert VIDEO_PATH or AUDIO_PATH, "Set VIDEO_PATH or AUDIO_PATH before running."
if VIDEO_PATH:
    assert Path(VIDEO_PATH).exists(), f"Video file not found: {VIDEO_PATH}"
if AUDIO_PATH:
    assert Path(AUDIO_PATH).exists(), f"Audio file not found: {AUDIO_PATH}"


In [13]:
# Convert input to mono 16k MP3 for stable, small uploads
input_path = VIDEO_PATH if VIDEO_PATH else AUDIO_PATH

!ffmpeg -y -i "{input_path}" -vn -ac 1 -ar 16000 -c:a libmp3lame -b:a 64k "{SAFE_AUDIO}"
!ffprobe -hide_banner -i "{SAFE_AUDIO}"

Path(SAFE_AUDIO).exists(), f"Audio file not found: {SAFE_AUDIO}"


ffmpeg version 8.0.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --

(True, 'Audio file not found: audio_openai.mp3')

## 2) Chunk and transcribe with OpenAI

`gpt-4o-mini-transcribe` is a good default; for higher quality, try `gpt-4o-transcribe`.


In [14]:
# Split into 10-minute chunks
CHUNKS_DIR.mkdir(parents=True, exist_ok=True)
chunk_pattern = (CHUNKS_DIR / "chunk_%03d.mp3").as_posix()

!ffmpeg -y -i "{SAFE_AUDIO}" -f segment -segment_time {CHUNK_SECONDS} -c copy "{chunk_pattern}"

chunk_files = sorted(CHUNKS_DIR.glob("chunk_*.mp3"))
print(f"Chunks: {len(chunk_files)}")
for p in chunk_files[:5]:
    print(" -", p)


Chunks: 6
 - chunks\chunk_000.mp3
 - chunks\chunk_001.mp3
 - chunks\chunk_002.mp3
 - chunks\chunk_003.mp3
 - chunks\chunk_004.mp3


ffmpeg version 8.0.1-full_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 15.2.0 (Rev8, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-lcms2 --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-libdvdnav --enable-libdvdread --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libopenjpeg --enable-libquirc --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-liboapv --enable-libqrencode --enable-librav1e --enable-libsvtav1 --enable-libvvenc --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libvpx --

### Transcribe Chunks


In [15]:
TRANSCRIBE_MODEL = "gpt-4o-mini-transcribe"  # or "gpt-4o-transcribe"

chunk_files = sorted(CHUNKS_DIR.glob("chunk_*.mp3"))
assert chunk_files, "No chunk files found. Run the chunking cell first."

all_text = []
for i, path in enumerate(chunk_files, 1):
    with path.open("rb") as f:
        t = client.audio.transcriptions.create(
            model=TRANSCRIBE_MODEL,
            file=f,
            response_format="text",
        )
    print(f"Chunk {i}/{len(chunk_files)} done: {path.name}")
    all_text.append(t if isinstance(t, str) else getattr(t, "text", str(t)))

transcription = "\n".join(all_text)
Path("meeting_transcript.txt").write_text(transcription, encoding="utf-8")

print("Saved transcription to", Path("meeting_transcript.txt").resolve())
display(Markdown("### Transcription Preview\n\n" + transcription[:3000]))


Chunk 1/6 done: chunk_000.mp3
Chunk 2/6 done: chunk_001.mp3
Chunk 3/6 done: chunk_002.mp3
Chunk 4/6 done: chunk_003.mp3
Chunk 5/6 done: chunk_004.mp3
Chunk 6/6 done: chunk_005.mp3
Saved transcription to C:\Users\l\Documents\Meeting minutes\meeting_transcript.txt


### Transcription Preview

Okay, so here is uploading the command to link the Everycha. And then there's a part where I store a new Everycha. And then there's also a part where we are searching for Everycha's are related to the query. So okay, I'm all right. You can tell me what your understanding is. All right, so far we are on the same path. No, I'm trying to understand it. That's what I'm trying to understand. Oh, so basically, my thought was this. So we have a vector database, last one, this vector database on the far right. Yeah. So my thinking was, let's say, who okay, the kind of questions he wants to keep, is he happy, is he what what? Yeah. Yeah, that's the layout we're working on. If you can check. Okay, okay. So technically, in front of upload, let's see, okay. Let's see, I'll show you. That's what I'm trying to do. Okay. Yeah. Okay, okay. So technically, in front of upload. Let's see, okay. But depending on what you want to do. So, I mean, I usually when you participate, not what you make. What's this? Out there starts to make it. Then you have to create another server. Another thing, what you may call it. Terminal. There should be a command for running that CRUD app. Just go up there. It should be up there. Okay, let me look for you. Okay, now, it see. There we go.

Sure, so we just have to upload maybe that document that you want. So you just pick time, boy. Why are you trying to inspect? What are you doing now? Oh, I'm gonna check the request. Let's go. Let's go. No, I'm trying to follow up my network request. What do I have? Okay, this is the upload, right? Yes, sir. And before. Okay. Is the Google. Oh, I can't run my API, right? What are my funds? I'm a little bit. I'm sure you, yeah. Go to, let's check it building. Down there, there's down, down, go to building. I think that's the error I'm reading. Yeah. That's what I'm trying to see. I'm sure that's what we agreed. So with the, that's the issue. Where are we storing, where are we persisting it? Where are we using it? That's why I'm connected. Okay, okay, let's roll, let's roll. Take me through again. Okay, so can you see? I can see, yes. Okay, let me look for a document to upload. I don't know. Can I send you a document? Send me a document by ChatPro. Chat. Oh, I don't see. Yeah, you can send to me. I don't know what. By Chat, I can send a document. I don't know. Send me that document. It's fine. Oh, no problem. Chief, what kind of technology are you on? Chief Advanced. But I think it Microsoft Teams can do that. Okay, let, now I contract. It's fine. Let's see, I think. Let's read what these guys want to say. Now I'm interested. Okay. That one would be. Okay. Let's see what happens here. All right. Let's see. So make it a surprise. See, I haven't run this. Let's see what happens. There we go. What is the primary role of the? So this is. Let's, we are finding out about this role. Okay, then how do you store the data? You download the JSON. Download JSON. There you go. Wait, can you open the JSON file? Th

In [16]:
# Optional: inspect transcript length
print(f"Transcript chars: {len(transcription):,}")


Transcript chars: 21,837


## 3) Generate meeting minutes (Markdown)

We?ll use the Responses API.


In [17]:
MINUTES_MODEL = "gpt-4.1-mini"  # or "gpt-5-mini" / "gpt-4.1" depending on your needs

instructions = (
    "You produce minutes of meetings from transcripts, with a summary, key discussion points, "
    "takeaways, and action items with owners, in markdown format without code blocks."
)

prompt = f"""Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

resp = client.responses.create(
    model=MINUTES_MODEL,
    instructions=instructions,
    input=prompt,
)

minutes_md = resp.output_text
display(Markdown(minutes_md))


# Denver Council Meeting Minutes

**Date:** [Date not specified in transcript]  
**Location:** Denver Council Meeting (virtual platform)  
**Attendees:** Multiple council members and participants (names not specified)  

---

## Summary

The meeting focused on the development and refinement of a software platform involving the upload, generation, curation, and publishing of educational content such as quizzes, flashcards, and summaries. Attendees discussed the current state of the backend, user interface, data persistence, and the workflow for content curation and publishing. Technical issues related to API usage, JSON file handling, and security concerns were also raised.

---

## Key Discussion Points

- Review of the system components including a vector database used for storing and searching educational content (Everycha).
- Challenges in uploading documents, tracking network requests, and error handling.
- Examination and understanding of JSON payload files generated for flashcards, quizzes, and summaries.
- Current functionality limited to generating one quiz or content type at a time versus generating multiple simultaneously.
- Workflow for content curation where teachers can select, modify, and keep preferred generated questions.
- The addition of an extra layer in the backend to allow selection and modification without changing the underlying data structure.
- Discussion on finalizing the summary format, preferring paragraph-based summaries over bullet points.
- Deployment considerations including backend completion, API setup (dummy publish API and creation API), and securing access via API keys.
- Clarification on frontend usage and user interfaces, referencing a simple app and lessons from existing platforms.
- Emphasis on maintaining the contract and structure to avoid breaking existing functionalities.
- Strategy for tracking generated content to maintain history and assist future use.
- Plans for publishing curated content and generating course or document metadata.
- General encouragement to complete the work while keeping the system stable and maintainable.

---

## Takeaways

- The platform is nearing backend completion with main functionalities in place for generating educational content.
- The focus now shifts towards refining curation capabilities and summary formatting.
- Security (API key management) and data persistence are critical for stable deployment.
- The current user approach involves selecting favorite generated questions and publishing curated results.
- The frontend is intended to be simple initially, with potential enhancements later.
- Maintaining compatibility with existing data structures is important for smooth integration.
- Collaboration and communication remain key to resolving outstanding issues and finalizing deliverables.

---

## Action Items

| Action Item                                                                                              | Owner/Responsible                     | Deadline/Status      |
|--------------------------------------------------------------------------------------------------------|-------------------------------------|---------------------|
| Fix summary generation to produce paragraph-style summaries instead of bullet points                    | Development Team                     | In progress         |
| Implement an extra backend layer for selecting and modifying generated questions without data structure changes | Development Team                     | In progress         |
| Build dummy publish API and creation API to enable content tracking and publishing                      | Development Team Lead                | Planned             |
| Establish secure API key management to restrict platform access                                        | Security Team / Development Team    | Planned             |
| Finalize understanding and handling of JSON payloads for uploaded and generated educational content     | All relevant developers             | Ongoing             |
| Improve document upload functionality and error debugging related to API and network requests           | Development Team                     | Ongoing             |
| Prepare simple frontend interface for initial platform use and demonstration                            | Frontend Developer / UI Designer    | Planned             |
| Maintain all social and technical contracts ensuring backward compatibility                             | Project Manager / Development Team  | Continuous          |
| Share sample documents and test files among team members for aligned development and testing            | All attendees                       | Immediate           |
| Complete content tracking system to keep history of generated content                                  | Development Team                     | Ongoing             |
| Schedule follow-up meeting to review progress and address any blockers                                 | Project Manager                     | TBD                 |

---

Minutes prepared by AI assistant based on meeting transcript extract.

## 4) Save minutes to a file


In [11]:
out_path = Path("meeting_minutes.md")
out_path.write_text(minutes_md, encoding="utf-8")
print(f"Saved: {out_path.resolve()}")


Saved: C:\Users\l\Documents\Meeting minutes\meeting_minutes.md
